## Imports

In [1]:
# Add project root to Python path ONLY FOR NOTEBOOK!!!
import sys
from pathlib import Path

project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [2]:
from neo4j import GraphDatabase
from qdrant_client import QdrantClient, models
from dotenv import load_dotenv
from pydantic import BaseModel
from openai import OpenAI
from collections import defaultdict
from neo4j_graphrag.retrievers import QdrantNeo4jRetriever
import uuid
import os

from config.datasets import GraphComponents, Single

In [3]:
# Load environment variables
load_dotenv()

True

---

## Setting Up Environment Variables

In [4]:
# Get credentials from environment variables (derived from docker-compose config)
qdrant_port = os.getenv("QDRANT_HTTP_PORT", "6333")
qdrant_url = f"http://localhost:{qdrant_port}"

neo4j_bolt_port = os.getenv("NEO4J_BOLT_PORT", "7687")
neo4j_uri = f"bolt://localhost:{neo4j_bolt_port}"

# Parse NEO4J_AUTH (format: username/password)
neo4j_auth = os.getenv("NEO4J_AUTH", "neo4j/password")
neo4j_username, neo4j_password = neo4j_auth.split("/", 1)

# LLM:
llm_model = os.getenv("LLM_MODEL")
llm_api_key = os.getenv("LLM_API_KEY")

In [5]:
# Print out all envs:
print(qdrant_url)
print(neo4j_uri)
print(neo4j_username)
print(neo4j_password)
print(llm_model)


http://localhost:6333
bolt://localhost:7687
neo4j
password
perplexity/sonar-pro


---

## Initialization of Neo4j and Qdrant Clients

In [6]:
neo4j_driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_username, neo4j_password))

qdrant_client = QdrantClient(
    url=qdrant_url,
)

---
## TEST

In [7]:
from ingestion.orchestration import Orchestrator

orchestrator = Orchestrator(llm_model, llm_api_key)
orchestrator._test_response("What is the capital of France?")

'The capital of France is **Paris**[1][2][3].\n\nParis has held the status of France’s capital almost continuously since the late 10th century, though there were short periods in history when the capital moved elsewhere, such as to Versailles or Vichy, due to political upheaval or war[5][6]. Paris is the largest city in France and is known as a major center for culture, commerce, and politics[1][3]. The city is located in the north-central part of the country along the Seine River[1].'

In [8]:
testing_graph =orchestrator.llm_parser(
    "Zachary has 3 children: Refael (male), his sister Noah, and the youngest boy Beerih."
)

In [9]:
testing_graph

GraphComponents(graph=[Single(node='Zachary', relationship='parent of', target_node='Refael'), Single(node='Zachary', relationship='parent of', target_node='Noah'), Single(node='Zachary', relationship='parent of', target_node='Beerih'), Single(node='Refael', relationship='sibling of', target_node='Noah'), Single(node='Refael', relationship='sibling of', target_node='Beerih'), Single(node='Noah', relationship='sibling of', target_node='Refael'), Single(node='Noah', relationship='sibling of', target_node='Beerih'), Single(node='Noah', relationship='sister of', target_node='Refael'), Single(node='Beerih', relationship='sibling of', target_node='Refael'), Single(node='Beerih', relationship='sibling of', target_node='Noah'), Single(node='Beerih', relationship='youngest child of', target_node='Zachary'), Single(node='Beerih', relationship='male', target_node=''), Single(node='Refael', relationship='male', target_node=''), Single(node='Noah', relationship='female', target_node='')])

In [10]:
testing_graph.graph

[Single(node='Zachary', relationship='parent of', target_node='Refael'),
 Single(node='Zachary', relationship='parent of', target_node='Noah'),
 Single(node='Zachary', relationship='parent of', target_node='Beerih'),
 Single(node='Refael', relationship='sibling of', target_node='Noah'),
 Single(node='Refael', relationship='sibling of', target_node='Beerih'),
 Single(node='Noah', relationship='sibling of', target_node='Refael'),
 Single(node='Noah', relationship='sibling of', target_node='Beerih'),
 Single(node='Noah', relationship='sister of', target_node='Refael'),
 Single(node='Beerih', relationship='sibling of', target_node='Refael'),
 Single(node='Beerih', relationship='sibling of', target_node='Noah'),
 Single(node='Beerih', relationship='youngest child of', target_node='Zachary'),
 Single(node='Beerih', relationship='male', target_node=''),
 Single(node='Refael', relationship='male', target_node=''),
 Single(node='Noah', relationship='female', target_node='')]

In [11]:
for entry in testing_graph.graph:
        node = entry.node
        target_node = entry.target_node  # Get target node if available
        relationship = entry.relationship  # Get relationship if available
        print(f"NODE: {node}, TARGET NODE: {target_node}, RELATIONSHIP: {relationship}")

NODE: Zachary, TARGET NODE: Refael, RELATIONSHIP: parent of
NODE: Zachary, TARGET NODE: Noah, RELATIONSHIP: parent of
NODE: Zachary, TARGET NODE: Beerih, RELATIONSHIP: parent of
NODE: Refael, TARGET NODE: Noah, RELATIONSHIP: sibling of
NODE: Refael, TARGET NODE: Beerih, RELATIONSHIP: sibling of
NODE: Noah, TARGET NODE: Refael, RELATIONSHIP: sibling of
NODE: Noah, TARGET NODE: Beerih, RELATIONSHIP: sibling of
NODE: Noah, TARGET NODE: Refael, RELATIONSHIP: sister of
NODE: Beerih, TARGET NODE: Refael, RELATIONSHIP: sibling of
NODE: Beerih, TARGET NODE: Noah, RELATIONSHIP: sibling of
NODE: Beerih, TARGET NODE: Zachary, RELATIONSHIP: youngest child of
NODE: Beerih, TARGET NODE: , RELATIONSHIP: male
NODE: Refael, TARGET NODE: , RELATIONSHIP: male
NODE: Noah, TARGET NODE: , RELATIONSHIP: female


---